In [ ]:
# Import required packages
import numpy as np
import pandas as pd
from time import time
from tabulate import tabulate

from paik.solver import (
    DEFAULT_SOLVER_PARAM_M7_NORM,
    max_joint_angle_change,
)

from path_follower import PathFollower

ddjc_thres = [40, 50, 60, 70, 80, 90, 100]

solver = PathFollower(solver_param=DEFAULT_SOLVER_PARAM_M7_NORM)

num_traj = 500
load_time = "1124134231"
J, P = solver.sample_Jtraj_Ppath(load_time=load_time, num_steps=20)
begin_time = time()
J_hat, l2_errs, ang_errs, mjac_arr, ddjc = solver.solve_path(J, P, num_traj=num_traj, return_numpy=True, return_evaluation=True)
end_time = time()

In [ ]:
df = pd.DataFrame(
    {
        "l2_errs": l2_errs,
        "ang_errs": np.rad2deg(ang_errs),
        "mjac": mjac_arr,
        "ddjc": np.rad2deg(ddjc),
    }
)

table = np.empty((len(ddjc_thres), 3))
for i, thres in enumerate(ddjc_thres):
    sr = df.query(f"ddjc < {thres}")["ddjc"].count() / df.shape[0]
    table[i] = (thres, sr, round(1 / sr)) if sr != 0 else (thres, sr, np.inf)
print(tabulate(table, headers=["ddjc", "success rate", "min num sols"]))
print(f"avg_inference_time: {round((end_time - begin_time) / num_traj, 3)}")
df.describe()

In [ ]:
import numpy as np
import pandas as pd
from time import time
import torch
from tqdm import trange
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver
from paik.solver import max_joint_angle_change

set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

In [ ]:
num_traj = 1000
num_steps = P.shape[0]

F_l2_err = np.empty((num_traj,))
F_ang_err = np.empty((num_traj,))
F_mjac = np.empty((num_traj,))
F_ddjc = np.empty((num_traj,))
F_runtime = np.empty((num_traj))

shape = (P.shape[0], ik_solver._network_width)
begin_time = time()
for i in trange(num_traj):
    latent = torch.from_numpy(
        np.tile(np.random.randn(1, shape[1]), (shape[0], 1)).astype(np.float32)
    ).to("cuda")
    sol, l2, ang, _, _, runtime = ik_solver.solve_n_poses(
        P, latent=latent, refine_solutions=False, return_detailed=True
    )  # type: ignore
    sol = sol.cpu().numpy()
    F_l2_err[i] = l2.mean()
    F_ang_err[i] = ang.mean()
    F_mjac[i] = max_joint_angle_change(sol)  # type: ignore
    F_ddjc[i] = np.linalg.norm(sol - J, axis=-1).mean()
    F_runtime[i] = runtime
end_time = time()
F_df = pd.DataFrame(
    {
        "l2_err": F_l2_err,
        "ang_err": np.rad2deg(F_ang_err),
        "mjac": F_mjac,
        "ddjc": np.rad2deg(F_ddjc),
    }
)

In [ ]:
table = np.empty((len(ddjc_thres), 3))
for i, thres in enumerate(ddjc_thres):
    sr = F_df.query(f"ddjc < {thres}")["ddjc"].count() / F_df.shape[0]
    table[i] = (thres, sr, round(1 / sr))
print(tabulate(table, headers=["ddjc", "success rate", "min num sols"]))
print(f"avg_inference_time: {round((end_time - begin_time) / num_traj, 3)}")
F_df.describe()